# Setup

In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
from collections import defaultdict
from collections.abc import Callable
from dataclasses import dataclass
import json
from itertools import combinations_with_replacement
import numpy as np
import os
import pandas as pd
import plotly.express as px
import plotly.graph_objects as go
from scipy.stats import chi2
from sklearn.model_selection import KFold
import statsmodels.api as sm
from tqdm import tqdm

from data import *
from plotting import *
from regression import *
from utils import *

In [3]:
results_dir = 'results/compute/with_speculative_compute/'
os.makedirs(results_dir, exist_ok=True)

In [4]:
colors = {'open': '#1f77b4', 'closed': '#ff7f0e'}

# Data preparation

In [5]:
# Load data
pcd_df = load_pcd_df()

In [6]:
pcd_df

,System,Domain,Task,Authors,Notability criteria,Notability criteria notes,Model accessibility,Link,Citations,Reference,...,Sparsity,Utilization notes,Estimated over 1e25 FLOP,Power per GPU,Cluster total TDP,Base model compute,Difference,Difference (share),API prices,Created
0,babbage-002,Language,Language modelling,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,0,NaN,0.000000e+00,NaN,"""$1.6 / 1M input tokens, fine tuned model"",""$1...",8/16/2024 1:26pm
1,tts-1,Speech,Text to Speech,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,0,NaN,0.000000e+00,NaN,$15.000 / 1M characters,8/16/2024 2:08pm
2,tts-1-hd,Speech,Text to Speech,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,0,NaN,0.000000e+00,NaN,$30.000 / 1M characters,8/16/2024 2:09pm
3,Theseus,Robotics,Maze solving,Claude Shannon,Historical significance,NaN,NaN,https://www.technologyreview.com/2018/12/19/13...,0.0,Mighty Mouse,...,NaN,NaN,NaN,NaN,0,NaN,4.000000e+01,100%,NaN,5/29/2023 2:06pm
4,SNARC,Robotics,Maze solving,Marvin Minsky,Historical significance,NaN,NaN,https://en.wikipedia.org/wiki/Stochastic_neura...,33.0,A Neural-Analogue Calculator Based upon a Prob...,...,NaN,NaN,NaN,NaN,0,NaN,0.000000e+00,NaN,NaN,5/29/2023 2:06pm
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1641,Flux.1 [dev],Image generation,Image generation,"Andreas Blattmann, Axel Sauer, Dominik Lorenz,...",NaN,Plausibly significant usage.\n\nCurrently the ...,Open access (non-commercial),https://blackforestlabs.ai/announcing-black-fo...,NaN,Flux.1,...,NaN,NaN,NaN,NaN,0,NaN,0.000000e+00,NaN,NaN,8/12/2024 4:13pm
1642,EXAONE 3.0,Language,"Language modelling/generation,Code generation,...","LG AI Research: Soyoung An, Kyunghoon Bae, Eun...",NaN,NaN,Open access (non-commercial),https://arxiv.org/abs/2408.03541,NaN,EXAONE 3.0 7.8B Instruction Tuned Language Model,...,NaN,NaN,NaN,700.0,0,NaN,4.000000e+23,100%,NaN,8/19/2024 10:33am
1643,Cosine Genie,Language,Code generation,Alistair Pullen,NaN,NaN,Unreleased,https://cosine.sh/blog/genie-technical-report,NaN,Genie is the best software engineering model i...,...,NaN,NaN,NaN,NaN,0,NaN,0.000000e+00,NaN,NaN,8/16/2024 6:04pm
1644,Grok-2,"Language,Vision,Multimodal","Chat,Language modelling/generation,Question an...",NaN,NaN,NaN,Hosted access (no API),https://x.ai/blog/grok-2,NaN,Grok-2 Beta Release,...,NaN,NaN,NaN,NaN,0,NaN,0.000000e+00,NaN,NaN,8/16/2024 8:34pm


In [7]:
pcd_df.loc[pcd_df['System'] == 'Megatron-BERT']['Model accessibility']

690    Unreleased
Name: Model accessibility, dtype: object

In [8]:
access_df = pcd_df.dropna(subset=['Publication date'])
len(access_df)

1643

In [9]:
access_df['Model accessibility'].unique()

array([nan, 'Unreleased', 'Open source', 'Hosted access (no API)',
       'Open access (non-commercial)', 'API access',
       'Open access (restricted use)'], dtype=object)

In [10]:
for cat in access_df['Model accessibility'].unique():
    if pd.isna(cat):
        print(cat, len(access_df.loc[access_df['Model accessibility'].isna()]))
    else:
        print(cat, len(access_df.loc[access_df['Model accessibility'] == cat]))

nan 611
Unreleased 463
Open source 318
Hosted access (no API) 28
Open access (non-commercial) 91
API access 59
Open access (restricted use) 73


In [11]:
open_access_categories = ['Open source', 'Open access (restricted use)', 'Open access (non-commercial)']
closed_access_categories = ['API access', 'Hosted access (no API)', 'Unreleased']

In [12]:
def get_access_label(access_category):
    if pd.isna(access_category):
        return 'Unknown'
    elif access_category in open_access_categories:
        return 'Open'
    elif access_category in closed_access_categories:
        return 'Closed'
    else:
        return 'Unknown'

# Add column with binary access label
access_df.loc[:, 'Model open/closed'] = access_df['Model accessibility'].apply(
    lambda x: get_access_label(x)
)

/var/folders/s2/stl5t_bn7xg9vdfxxr71rzrc0000gn/T/ipykernel_32096/3052041378.py:12: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  access_df.loc[:, 'Model open/closed'] = access_df['Model accessibility'].apply(


In [13]:
access_df

,System,Domain,Task,Authors,Notability criteria,Notability criteria notes,Model accessibility,Link,Citations,Reference,...,Utilization notes,Estimated over 1e25 FLOP,Power per GPU,Cluster total TDP,Base model compute,Difference,Difference (share),API prices,Created,Model open/closed
3,Theseus,Robotics,Maze solving,Claude Shannon,Historical significance,NaN,NaN,https://www.technologyreview.com/2018/12/19/13...,0.0,Mighty Mouse,...,NaN,NaN,NaN,0,NaN,4.000000e+01,100%,NaN,5/29/2023 2:06pm,Unknown
4,SNARC,Robotics,Maze solving,Marvin Minsky,Historical significance,NaN,NaN,https://en.wikipedia.org/wiki/Stochastic_neura...,33.0,A Neural-Analogue Calculator Based upon a Prob...,...,NaN,NaN,NaN,0,NaN,0.000000e+00,NaN,NaN,5/29/2023 2:06pm,Unknown
5,Genetic algorithm,Mathematics,Numerical simulation,NA Barricelli,Historical significance,Possibly first computer simulation of a geneti...,NaN,https://link.springer.com/article/10.1007/BF01...,266.0,Numerical testing of evolution theories,...,NaN,NaN,NaN,0,NaN,0.000000e+00,NaN,NaN,5/29/2023 2:06pm,Unknown
6,Sequence-based pattern recognition,Vision,Character recognition,O. G. Selfridge,Historical significance,NaN,NaN,https://dl.acm.org/doi/10.1145/1455292.1455310,290.0,Pattern recognition and modern computers,...,NaN,NaN,NaN,0,NaN,0.000000e+00,NaN,NaN,5/29/2023 2:06pm,Unknown
7,Self Organizing System,Other,Pattern recognition,W. A. Clark and B. G. Farley,Historical significance,NaN,NaN,https://dl.acm.org/doi/10.1145/1455292.1455309,93.0,Generalization of pattern recognition in a sel...,...,NaN,NaN,NaN,0,NaN,0.000000e+00,NaN,NaN,5/29/2023 2:06pm,Unknown
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1641,Flux.1 [dev],Image generation,Image generation,"Andreas Blattmann, Axel Sauer, Dominik Lorenz,...",NaN,Plausibly significant usage.\n\nCurrently the ...,Open access (non-commercial),https://blackforestlabs.ai/announcing-black-fo...,NaN,Flux.1,...,NaN,NaN,NaN,0,NaN,0.000000e+00,NaN,NaN,8/12/2024 4:13pm,Open
1642,EXAONE 3.0,Language,"Language modelling/generation,Code generation,...","LG AI Research: Soyoung An, Kyunghoon Bae, Eun...",NaN,NaN,Open access (non-commercial),https://arxiv.org/abs/2408.03541,NaN,EXAONE 3.0 7.8B Instruction Tuned Language Model,...,NaN,NaN,700.0,0,NaN,4.000000e+23,100%,NaN,8/19/2024 10:33am,Open
1643,Cosine Genie,Language,Code generation,Alistair Pullen,NaN,NaN,Unreleased,https://cosine.sh/blog/genie-technical-report,NaN,Genie is the best software engineering model i...,...,NaN,NaN,NaN,0,NaN,0.000000e+00,NaN,NaN,8/16/2024 6:04pm,Closed
1644,Grok-2,"Language,Vision,Multimodal","Chat,Language modelling/generation,Question an...",NaN,NaN,NaN,Hosted access (no API),https://x.ai/blog/grok-2,NaN,Grok-2 Beta Release,...,NaN,NaN,NaN,0,NaN,0.000000e+00,NaN,NaN,8/16/2024 8:34pm,Closed


In [14]:
print('Closed', len(access_df[access_df['Model open/closed'] == 'Closed']))
print('Open', len(access_df[access_df['Model open/closed'] == 'Open']))
print('Unknown', len(access_df[access_df['Model open/closed'] == 'Unknown']))

Closed 550
Open 482
Unknown 611


In [15]:
df = access_df

# [parameters]

In [16]:
# 'external': Filter to the top n models overall
# 'internal': Filter to the top n models within 'Open' and 'Closed' categories
# 'disabled': No filtering
frontier_selection = 'external'  # ['disabled', 'internal', 'external']
top_n = 10  # Filter to the top n models by training compute at time of release
model_selection = 'All models'  # ['All models', 'Language models', 'Google DeepMind models', 'OpenAI models', 'Meta AI models']
filter_alphago_outliers = True
filter_finetuned_models = True
include_speculative_compute = True
cutoff_date = '2018-01-01'
top_n_cutoff_date = '2018-01-01'

# end [parameters]

In [17]:
def find_top_models_up_to_release(df, top_n):
    """Find the models which were in the top n by compute when they were released."""
    # This set will keep track of models that were ever in the top 10 at their release
    ever_in_top_n = set()

    # Iterate over each date in the DataFrame
    for current_date in df['date'].unique():
        # Get all entries up to the current date
        historical_data = df[df['date'] <= current_date]
        # Find top 10 models by flop count in this subset
        top_n_models = historical_data.nlargest(top_n, 'flop')['System']
        # Update the set of models that were ever in top n
        ever_in_top_n.update(top_n_models)

    # Return DataFrame filtered to only include models that were ever in the top 10
    return df[df['System'].isin(ever_in_top_n)]


def filter_top_models_within_category(df, top_n, cutoff_date, category):
    """Find the models which were in the top-n by compute when they were released,
    among models in the specified category. The top-n models in the specified category
    are seeded with the overall top-n models before the cutoff date.
    """
    # Filter top-n models within the category, but seeded with overall top-n models
    top_models_df = find_top_models_up_to_release(df, top_n)
    top_n_models_at_cutoff_date_df = top_models_df[top_models_df['date'] <= cutoff_date].nlargest(top_n, 'flop')
    category_df = df[df['category'] == category]

    # This set will keep track of models that were ever in the top 10 at their release
    ever_in_top_n = set()

    # Iterate over each date in the DataFrame
    for current_date in category_df['date'].unique():
        # if current_date < pd.to_datetime(cutoff_date):
        #     continue
        # Get all entries up to the current date
        historical_data = pd.concat([category_df[category_df['date'] <= current_date], top_n_models_at_cutoff_date_df])
        # Find top 10 models by flop count in this subset
        top_n_models_df = historical_data.nlargest(top_n, 'flop')
        # Update the set of models that were ever in top n
        # Filter out the models that aren't in the category
        ever_in_top_n.update(top_n_models_df[top_n_models_df['category'] == category]['System'])

    # Return DataFrame filtered to only include models that were ever in the top 10
    return df[df['System'].isin(ever_in_top_n)]


def filter_top_models_in_both_categories(df, top_n, cutoff_date):
    # Get top models for Open and Closed categories
    top_open_models = filter_top_models_within_category(df, top_n, cutoff_date, category='Open')
    top_closed_models = filter_top_models_within_category(df, top_n, cutoff_date, category='Closed')
    # Combine the results
    df_filtered = pd.concat([top_open_models, top_closed_models])
    # Sort the combined DataFrame by date
    df_filtered = df_filtered.sort_values('date')
    return df_filtered

In [18]:
df_filtered = (df[['System', 'Training compute (FLOP)', 'Publication date', 'Organization', 'Notability criteria', 'Domain', 'Base model', 'Model open/closed']]
    .rename(columns={'Training compute (FLOP)': 'flop', 'Publication date': 'date', 'Model open/closed': 'category'})
    .assign(date=lambda x: pd.to_datetime(x['date']), log_flop=lambda x: np.log10(x['flop']))
    .sort_values('date'))
list(df_filtered[df_filtered['Base model'].notna()]['System'])

['SSD',
 'Layer Normalization: Handwriting sequence generation',
 'Layer Normalization: Skip Thoughts',
 'Layer Normalization: Draw',
 'Layer Normalization: Order embeddings of images and language',
 'Layer Normalization: The Attentive Reader',
 'ULM-FiT',
 'ADP-FAIRSEQ + NGRAMRES',
 'Cross-lingual alignment',
 'Theseus 6/768',
 'UnifiedQA',
 'GPT-Neo-2.7B (finetuned)',
 'GPT-Neo-2.7B (finetuned on PTB)',
 'Unicorn',
 'Multitask Unified Model (MUM)',
 '$\\infty$-former (SM)',
 'FLAN 137B',
 'AlphaFold-Multimer',
 'GPT-2 (AMPS)',
 'Masked Autoencoders',
 'Engine-XL(NE)',
 'HSO',
 'Contriever',
 'Vespa',
 'OntoProtein',
 'InstructGPT',
 'BERT-RBP',
 'Flamingo',
 'Jurassic-X',
 'SimCSE',
 'CogVideo',
 'Minerva (540B)',
 'Delphi',
 'Transformer-XL + RMT',
 'GPT-NeoX-Japanese',
 'BlenderBot 3',
 'PaLM-SayCan',
 'Sparrow',
 'NMST+GPT-2',
 'Decaying Fast Weights Transformer (WT-103)',
 "Instruct-GPT + Mind's Eye",
 'GPT-2 + Progressive LRD',
 'Flan-T5 11B',
 'Flan-PaLM 540B',
 'LMSI-Palm',
 '

In [19]:
df_filtered = (df[['System', 'Training compute (FLOP)', 'Publication date', 'Organization', 'Notability criteria', 'Domain', 'Base model', 'Model open/closed']]
    .rename(columns={'Training compute (FLOP)': 'flop', 'Publication date': 'date', 'Model open/closed': 'category'})
    .assign(date=lambda x: pd.to_datetime(x['date']), log_flop=lambda x: np.log10(x['flop']))
    .sort_values('date'))

# Add speculative compute estimates based on benchmark imputation and rough guesses
if include_speculative_compute:
    speculative_compute_estimates = {
        "Claude 3.5 Sonnet": 5.0e25,
        "Claude 3 Opus": 2.5e25,
        "Claude 3 Sonnet": 1.1e25,
        "GPT-4o": 2.9e25,
        "Gemini 1.0 Pro": 2.8e24,
        "Gemini 1.5 Pro": 1.9e25,
        "Reka Core": 8.4e24,
        "GPT-4 Turbo": 2.1e25,  # rough guess
        "GPT-4V": 2.1e25,  # rough guess
        "Claude 2.1": df_filtered[df_filtered["System"]=="Claude 2"]["flop"].values,  # rough guess
    }
    for model, compute in speculative_compute_estimates.items():
        df_filtered.loc[df_filtered["System"] == model, "flop"] = compute
        df_filtered.loc[df_filtered["System"] == model, "log_flop"] = np.log10(compute)

df_filtered.dropna(subset=['flop'], inplace=True)

# Drop Alpha Go Master / Zero
if filter_alphago_outliers:
    mask = (df_filtered["System"] == 'AlphaGo Master') | (df_filtered["System"] == 'AlphaGo Zero')
    df_filtered = df_filtered[~mask]

# Drop finetuned models
if filter_finetuned_models:
    mask = df_filtered['Base model'].isna()
    df_filtered = df_filtered[mask]

top_models_df = find_top_models_up_to_release(df_filtered, top_n)  # For reference

if frontier_selection == 'external':
    # Filter top models before other filters
    df_filtered = filter_top_models_in_both_categories(df_filtered, top_n, top_n_cutoff_date)

if model_selection == 'Language models':
    re = 'Language|Multimodal'
    mask = df_filtered['Domain'].str.contains(re, na=False)
    df_filtered = df_filtered[mask]

if frontier_selection == 'internal':
    # Filter top models after other filters
    df_filtered = filter_top_models_in_both_categories(df_filtered, top_n, top_n_cutoff_date)

# Filter for models after the cutoff date
df_filtered = df_filtered[df_filtered['date'] > cutoff_date]

print(f"{len(df_filtered)}{' top' if frontier_selection != 'disabled' else ''} {model_selection} models found")
print(f"They span {df_filtered['date'].min().strftime('%B %Y')} to {df_filtered['date'].max().strftime('%B %Y')}")

122 top All models models found
They span February 2018 to July 2024


In [20]:
open_df = df_filtered[df_filtered['category'] == 'Open']
closed_df = df_filtered[df_filtered['category'] == 'Closed']
recent_top_models_df = top_models_df[top_models_df['date'] > pd.to_datetime('2010-01-01')]

fig = go.Figure()

fig.add_trace(go.Scatter(
    x=open_df['date'],
    y=open_df['log_flop'],
    mode='markers',
    marker=dict(color=colors['open'], opacity=0.5),
    text=open_df['System'],
    hoverinfo='text',
    name=f'Top-{top_n} Open'
))

fig.add_trace(go.Scatter(
    x=closed_df['date'],
    y=closed_df['log_flop'],
    mode='markers',
    marker=dict(color=colors['closed'], opacity=0.5),
    text=closed_df['System'],
    hoverinfo='text',
    name=f'Top-{top_n} Closed'
))

fig.add_trace(go.Scatter(
    x=recent_top_models_df['date'],
    y=recent_top_models_df['log_flop'],
    mode='markers',
    marker=dict(color='grey', opacity=0.5),
    text=recent_top_models_df['System'],
    hoverinfo='text',
    name=f'Top-{top_n} Overall'
))

fig.update_layout(
    width=800,
    height=400,
    xaxis_title='Date',
    yaxis_title='Log FLOP',
    title=f'Top-{top_n} models with kickstarting',
    margin=dict(t=50, l=60, r=60, b=50),
)

# save_plot(fig, results_dir, f'top_{top_n}_models_without_kickstarting')

fig.show()

In [21]:
# Measure the lag in compute between top-1 models in open and closed categories
lags = []
already_matched = set()
for i, closed_row in closed_df.iterrows():
    for j, open_row in open_df.iterrows():
        if open_row['log_flop'] >= closed_row['log_flop'] and open_row['date'] not in already_matched:
            lag_months = (open_row['date'] - closed_row['date']).days/365*12
            print(f"{open_row['System']} exceeded {closed_row['System']} after {lag_months:.1f} months")
            # already_matched.add(open_row['date'])
            lags.append(lag_months)
            break
lags

ResNeXt-101 32x48d exceeded AmoebaNet-A (F=448) after 2.8 months
ResNeXt-101 32x48d exceeded FTW after -2.0 months
ResNeXt-101 32x48d exceeded BERT-Large-CAS (PTB+WT2+WT103) after -11.6 months
T5-11B exceeded Megatron-LM (8.3B) after 1.2 months
T5-11B exceeded Megatron-BERT after 1.2 months
ResNeXt-101 32x48d exceeded Rubik's cube ADR robot after -17.5 months
mT5-XXL exceeded AlphaStar after 11.7 months
mT5-XXL exceeded OpenAI Five after 10.3 months
T5-11B exceeded OpenAI Five Rerun after -1.7 months
ResNeXt-101 32x48d exceeded ContextNet + Noisy Student after -20.6 months
OPT-175B exceeded Meena after 27.1 months
T5-11B exceeded Turing-NLG after -3.7 months
T5-11B exceeded GPT3-6.7B (rerun of original) after -7.2 months
OPT-175B exceeded GPT-3 175B (davinci) after 23.1 months
T5-11B exceeded GShard (600B) after -8.3 months
OPT-175B exceeded GShard (dense) after 22.1 months
mT5-XXL exceeded DALL-E after -2.5 months
mT5-XXL exceeded Meta Pseudo Labels after -4.3 months
mT5-XXL exceeded 

[2.8273972602739725,
 -2.0383561643835617,
 -11.605479452054794,
 1.1835616438356165,
 1.1835616438356165,
 -17.457534246575342,
 11.704109589041096,
 10.257534246575343,
 -1.6767123287671235,
 -20.613698630136987,
 27.12328767123288,
 -3.715068493150685,
 -7.167123287671233,
 23.145205479452056,
 -8.252054794520546,
 22.06027397260274,
 -2.5315068493150683,
 -4.33972602739726,
 -5.950684931506849,
 -6.147945205479452,
 -7.627397260273973,
 -9.205479452054794,
 8.67945205479452,
 7.693150684931507,
 22.84931506849315,
 6.641095890410958,
 19.2986301369863,
 4.602739726027397,
 20.745205479452054,
 20.449315068493153,
 2.926027397260274,
 2.663013698630137,
 2.432876712328767,
 7.364383561643836,
 17.0958904109589,
 4.997260273972603,
 -1.6767123287671235,
 9.271232876712329,
 -9.336986301369864,
 16.306849315068494,
 -4.33972602739726,
 13.183561643835617,
 2.4657534246575343,
 0.3945205479452054,
 9.271232876712329,
 9.928767123287672,
 6.673972602739727,
 -1.8082191780821917,
 8.5479

In [22]:
lags = np.array(lags)
lags.mean(), np.percentile(lags, [5, 95])

(3.9480894015861576, array([-9.79068493, 22.21808219]))

# Regression analysis

## Model selection

In [23]:
@dataclass
class FitResult:
    p: int = None
    bic: float = None
    rss: float = None
    mse: float = None
    predict: Callable = None


@dataclass
class KinkedFitResult(FitResult):
    break_points: tuple[float] = None
    break_points_dt: float = None
    oom_year_slopes: tuple[float] = None
    intercepts: tuple[float] = None

    # Model properties for each breakpoint combination
    # (for debugging)
    bics: tuple[float] = None
    rsss: tuple[float] = None
    mses: tuple[float] = None
    break_points_list: tuple[tuple[float]] = None
    break_points_dt_list: tuple[tuple[float]] = None


def get_predictors(
    x,
    intercept_change_points,
    slope_change_points,
    pred_category=None,category=None,
    same_intercepts=None,
    same_slopes=None
):
    if pred_category == 'Open':
        is_open = np.ones(len(x))
    elif pred_category == 'Closed':
        is_open = np.zeros(len(x))
    else:
        assert category is not None
        is_open = (category == 'Open').astype(int).values

    # Ensure the lengths match
    assert len(same_intercepts) == len(intercept_change_points), f"Length of same_intercepts ({len(same_intercepts)}) must match the number of intercept change points ({len(intercept_change_points)})"
    assert len(same_slopes) == len(slope_change_points), f"Length of same_slopes ({len(same_slopes)}) must match the number of slope change points ({len(slope_change_points)})"

    # Calculate the number of columns needed
    n_intercept_cols = sum(1 if same else 2 for same in same_intercepts)
    n_slope_cols = sum(1 if same else 2 for same in same_slopes)
    n_cols = n_intercept_cols + n_slope_cols

    predictors = np.zeros((len(x), n_cols))

    # Intercept predictors
    col_idx = 0
    for i, (intercept_point, same) in enumerate(zip(intercept_change_points, same_intercepts)):
        if same:
            predictors[:, col_idx] = (x >= intercept_point).astype(int)
            col_idx += 1
        else:
            predictors[:, col_idx] = (x >= intercept_point).astype(int) * is_open
            predictors[:, col_idx + 1] = (x >= intercept_point).astype(int) * (1 - is_open)
            col_idx += 2

    # Slope predictors
    for i, (break_point, same) in enumerate(zip(slope_change_points, same_slopes)):
        if same:
            predictors[:, col_idx] = np.maximum(x - break_point, 0)
            col_idx += 1
        else:
            predictors[:, col_idx] = np.maximum(x - break_point, 0) * is_open
            predictors[:, col_idx + 1] = np.maximum(x - break_point, 0) * (1 - is_open)
            col_idx += 2

    return predictors


def fit_n_phase_exponential(
    df,
    kink_count,
    allow_discontinuities=False,
    same_intercepts=None,
    same_slopes=None,
    min_n_segment=10
):
    # Generate monthly breakpoints between 2010 and 2024
    one_month = pd.DateOffset(months=1)
    break_point_grid = pd.date_range(start=df['date'].min() - one_month, end=df['date'].max() - 4*one_month, freq='MS')
    break_point_grid = [x.toordinal() for x in break_point_grid]

    x = pd.to_datetime(df['date']).apply(lambda date: date.toordinal()).values
    y = df['log_flop'].values

    break_points_list = []
    bics = []
    rsss = []
    mses = []
    models = []

    for break_points in combinations_with_replacement(break_point_grid, kink_count):
        intercept_change_points = (0,)
        if allow_discontinuities:
            intercept_change_points += break_points
        slope_change_points = (0,) + break_points

        # If same_intercepts or same_slopes are not provided, default to all False
        if same_intercepts is None:
            same_intercepts = [False] * len(intercept_change_points)
        if same_slopes is None:
            same_slopes = [False] * len(slope_change_points)

        predictors = get_predictors(
            x,
            intercept_change_points,
            slope_change_points,
            category=df['category'],
            same_slopes=same_slopes,
            same_intercepts=same_intercepts
        )

        # Fit the model
        model = sm.OLS(y, predictors).fit()

        # Calculate BIC manually based on log-likelihood
        n = len(x) # Number of observations
        p = len(model.params) + 2*kink_count + 1 # Number of parameters

        # Calculate log-likelihood under the assumption of normally distributed errors
        # We have to iterate over all points to get their individual log-likelihoods
        log_likelihood = 0
        rss = 0
        invalid_model = False # Discard models with segments with less than 2 points
        for i, break_point in enumerate(slope_change_points):
            left_x = break_point
            right_x = slope_change_points[i + 1] if i + 1 < len(slope_change_points) else np.inf

            segment_predictors = predictors[(left_x <= x) & (x < right_x), :]
            segment_y = y[(left_x <= x) & (x < right_x)]
            segment_n = len(segment_y)

            assert min_n_segment > 2

            if segment_n < min_n_segment:
                invalid_model = True
                break

            y_pred = model.predict(segment_predictors)

            segment_rss = np.sum((y_pred - segment_y)**2)
            assert segment_rss > 0
            segment_mse = segment_rss / segment_n

            segment_log_likelihood = -segment_n/2 * (np.log(2*np.pi) + np.log(segment_rss/segment_n) + 1)
            log_likelihood += segment_log_likelihood
            rss += segment_rss

        if invalid_model:
            continue

        # Compute BIC using the manual method based on the log-likelihood
        bic = p * np.log(n) - 2 * log_likelihood
        # bic = n*np.log(rss/n) + p*np.log(n)

        bics.append(bic)
        rsss.append(rss)
        mses.append(rss/len(df))
        models.append(model)
        break_points_list.append(break_points)

    # Prepare the result object
    best_bic = min(bics)
    best_idx = bics.index(best_bic)
    best_rss = rsss[best_idx]
    best_mse = mses[best_idx]
    best_model = models[best_idx]
    best_break_points = break_points_list[best_idx]

    p = len(best_model.params) + 2*kink_count + 1 # Number of parameters

    # Store the model parameters
    intercept_change_points = (0,)
    if allow_discontinuities:
        intercept_change_points += best_break_points
    slope_change_points = (0,) + best_break_points

    n_intercepts = sum(1 if same else 2 for same in same_intercepts)
    intercepts = best_model.params[:n_intercepts]
    oom_intercepts = np.zeros((2, len(intercept_change_points)))
    for i in range(len(intercept_change_points)):
        if same_intercepts[i]:
            oom_intercepts[0, i] = oom_intercepts[1, i] = intercepts[i]
        else:
            oom_intercepts[0, i] = intercepts[2*i - sum(same_intercepts[:i])]
            oom_intercepts[1, i] = intercepts[2*i + 1 - sum(same_intercepts[:i])]

    # Apply cumulative sum to get the actual slopes
    oom_intercepts = {'open': np.cumsum(oom_intercepts[0]), 'closed': np.cumsum(oom_intercepts[1])}

    n_slopes = len(slope_change_points)
    slopes = best_model.params[n_intercepts:]
    oom_year_slopes = np.zeros((2, n_slopes))  # 2 rows for Open and Closed
    for i in range(n_slopes):
        if same_slopes[i]:
            oom_year_slopes[0, i] = oom_year_slopes[1, i] = 365 * slopes[i]
        else:
            oom_year_slopes[0, i] = 365 * slopes[2*i - sum(same_slopes[:i])]
            oom_year_slopes[1, i] = 365 * slopes[2*i + 1 - sum(same_slopes[:i])]

    # Apply cumulative sum to get the actual slopes
    oom_year_slopes = {'open': np.cumsum(oom_year_slopes[0]), 'closed': np.cumsum(oom_year_slopes[1])}

    def predict(date, category):
        if not isinstance(date, pd.Series):
            date = pd.Series(date)
        x = pd.to_datetime(date).apply(lambda date: date.toordinal()).values

        predictors = get_predictors(
            x,
            intercept_change_points,
            slope_change_points,
            category=category,
            same_slopes=same_slopes,
            same_intercepts=same_intercepts
        )

        return best_model.predict(predictors)

    fit_result = KinkedFitResult(
        p=p,
        bic=best_bic,
        rss=best_rss,
        mse=best_mse,
        break_points=best_break_points,
        predict=predict,
        break_points_dt=[pd.Timestamp.fromordinal(bp) for bp in best_break_points],
        bics=bics,
        rsss=rsss,
        mses=mses,
        oom_year_slopes=oom_year_slopes,
        intercepts=oom_intercepts,
        break_points_list=break_points_list,
        break_points_dt_list=[[pd.Timestamp.fromordinal(bp) for bp in break_points] for break_points in break_points_list],
    )

    return fit_result


fit_em_all = lambda df_fit : {
    "Simple" : fit_n_phase_exponential(df_fit, kink_count=0),
    # "Simple with same slope": fit_n_phase_exponential(df_fit, kink_count=0, same_slopes=(True,)),
    # "Simple with same slope and intercept": fit_n_phase_exponential(df_fit, kink_count=0, same_slopes=(True,), same_intercepts=(True,)),
    # "Discrete acceleration" : fit_n_phase_exponential(df_fit, kink_count=1),
    # "Discontinuity" : fit_n_phase_exponential(df_fit, kink_count=1, allow_discontinuities=True),
    # "Same pre-break different post-break" : fit_n_phase_exponential(
    #     df_fit, kink_count=1, allow_discontinuities=True, same_intercepts=(True, False), same_slopes=(True, False)
    # ),
    # "Same pre-break different intercept post-break" : fit_n_phase_exponential(
    #     df_fit, kink_count=1, allow_discontinuities=True, same_intercepts=(True, False), same_slopes=(True, True)
    # ),
    # "Same pre-break and post-break" : fit_n_phase_exponential(
    #     df_fit, kink_count=1, allow_discontinuities=True, same_intercepts=(True, True), same_slopes=(True, True)
    # ),
}


# K-Fold Cross Validation
def perform_cross_validation(df, k=10, random_state=42):
    kf = KFold(n_splits=k, shuffle=True, random_state=random_state)
    folds_mses = defaultdict(lambda : [])
    for train_index, test_index in kf.split(df):
        train_df, test_df = df.iloc[train_index], df.iloc[test_index]

        # Fit the models on the training set
        fold_models = fit_em_all(train_df)

        # Predict on the test set
        for name,model in fold_models.items():
            try:
                predicted_log_flop = model.predict(test_df["date"], test_df["category"])
            except AttributeError:
                continue
            test_rss = np.sum((predicted_log_flop - test_df["log_flop"])**2)
            test_mse = test_rss / len(test_df)
            folds_mses[name].append(test_mse)

    # Compute mean MSE
    folds_mses = {name: np.mean(folds_mses[name]) for name in folds_mses}

    return folds_mses


def calculate_lead_time(df, fit_result):
    # Get the final predictions for 'open' and 'closed' categories
    final_date = df['date'].max()
    y_open = fit_result.predict(pd.Series([final_date]), pd.Series(['Open']))[0]
    y_closed = fit_result.predict(pd.Series([final_date]), pd.Series(['Closed']))[0]
    
    # Get the final slope for the 'closed' category
    slope_closed = fit_result.oom_year_slopes['closed'][-1]
    
    # Calculate lead time
    lead_time = (y_closed - y_open) / slope_closed
    
    return lead_time

In [24]:
# Best model fits
models = fit_em_all(df_filtered)

# K-fold cross validation
folds_mses = perform_cross_validation(df_filtered)

# Bootstrap
bootstrap_sample_size = 1000

rng = np.random.default_rng(43)
bootstrap_bics = defaultdict(lambda : [])
bootstrap_mses = defaultdict(lambda : [])
bootstrap_bic_score_diff = defaultdict(lambda : [])
bootstrap_slopes = defaultdict(lambda : defaultdict(lambda : []))
bootstrap_intercepts = defaultdict(lambda : defaultdict(lambda : []))
bootstrap_breaks = defaultdict(lambda : [])
bootstrap_lead_time_years = defaultdict(lambda : [])
for bootstrap_index in tqdm(range(bootstrap_sample_size)):
    sample = df_filtered.sample(len(df_filtered), replace=True, random_state=rng)
    sample = sample.sort_values('date')

    # Compute BICs
    boot_models = fit_em_all(sample)

    # Compute K fold validation
    boot_folds_mses = perform_cross_validation(sample)

    # Store results
    for name, model in boot_models.items():
        # It might be None if the hyperbolic fails to fit
        if model is None: continue

        bootstrap_bics[name].append(model.bic)
        bootstrap_mses[name].append(boot_folds_mses[name])
        bootstrap_bic_score_diff[name].append(model.bic - boot_models["Simple"].bic)

        if isinstance(model, KinkedFitResult):
            if (len(model.oom_year_slopes['open']) > 0):
                bootstrap_slopes[name]['open'].append(10**model.oom_year_slopes['open'][-1])
            if (len(model.oom_year_slopes['closed']) > 0):
                bootstrap_slopes[name]['closed'].append(10**model.oom_year_slopes['closed'][-1])
            if (len(model.break_points_dt) > 0):
                bootstrap_breaks[name].append(model.break_points_dt[-1])

            # Calculate the lead time between predictions for the open and closed categories
            lead_time_years = calculate_lead_time(sample, model)
            bootstrap_lead_time_years[name].append(lead_time_years)


ci_width = 0.90
qs = [(1 - ci_width)/2, (1 + ci_width)/2]
bootstrap_preferred_percent = {}
bootstrap_slopes_ci = defaultdict(lambda : defaultdict(lambda : []))
for name in models:
    bootstrap_preferred_percent[name] = np.mean(np.array(bootstrap_bic_score_diff[name])<0)
    bootstrap_bics[name] = np.quantile(np.array(bootstrap_bics[name]), qs)
    bootstrap_mses[name] = np.quantile(np.array(bootstrap_mses[name]), qs)
    bootstrap_bic_score_diff[name] = np.quantile(np.array(bootstrap_bic_score_diff[name]), qs)
    bootstrap_slopes_ci[name]['open'] = np.quantile(np.array(bootstrap_slopes[name]['open']), qs)
    bootstrap_slopes_ci[name]['closed'] = np.quantile(np.array(bootstrap_slopes[name]['closed']), qs)
    bootstrap_lead_time_years[name] = np.quantile(np.array(bootstrap_lead_time_years[name]), qs)
    if len(bootstrap_breaks[name]) > 0:
        bootstrap_breaks[name] = np.quantile(np.array(bootstrap_breaks[name]), qs)

# Models with lower BIC score / MSE are preferred.

results = []
for name, model in models.items():
    param_count = model.p
    log_likelihood = (np.log(len(df_filtered))*param_count - model.bic)/2

    param_count_simple = models['Simple'].p
    log_likelihood_simple = (np.log(len(df_filtered))*param_count_simple - models['Simple'].bic)/2

    c2 = chi2.sf(2*(log_likelihood - log_likelihood_simple), df=(param_count - param_count_simple))

    result = {
        "Model": name,
        "BIC" : np.round(model.bic, 2),
        "BIC 90% CI" : np.round(bootstrap_bics[name], 2),
        #"Parameter count": param_count,
        #"Log likelihood": np.round((np.log(len(df_filtered))*param_count - model.bic)/2),
        # "MSE" : model.mse,
        "BIC score diff": np.round(model.bic - models["Simple"].bic, 2),
        "BIC score diff 90% CI": np.round(bootstrap_bic_score_diff[name], 2),
        "Xi²": c2,
        "% times preferred over simple": f"{bootstrap_preferred_percent[name]:.0%}",
        # "bayes factor over simple" : np.exp(-0.5 * (model.bic - models["simple"].bic)),
        "K-fold mean MSE" : np.round(folds_mses[name], 2),
        "K-fold mean MSE 90% CI" : np.round(bootstrap_mses[name], 2),
    }

    result["Recent slope for closed models (Nx/year)"] = np.round(10**model.oom_year_slopes['closed'][-1], 2)
    result["Recent slope for closed models 90% CI"] = np.round(bootstrap_slopes_ci[name]['closed'], 2)
    result["Recent slope for open models (Nx/year)"] = np.round(10**model.oom_year_slopes['open'][-1], 2)
    result["Recent slope for open models 90% CI"] = np.round(bootstrap_slopes_ci[name]['open'], 2)
    result["Lead time (years)"] = np.round(calculate_lead_time(df_filtered, model), 2)
    result["Lead time 90% CI"] = np.round(bootstrap_lead_time_years[name], 2)
    if len(model.break_points_dt) > 0:
        result["Break point"] = model.break_points_dt[-1].strftime('%Y-%m')
        result["Break point 90% CI"] = [date.strftime('%Y-%m') for date in bootstrap_breaks[name]]
    results.append(result)

results_df = pd.DataFrame(results)

# bayes_factor = np.exp(-0.5 * (kinked_fit.bic - simple_fit.bic))

print("Results")
results_df

100%|██████████| 1000/1000 [00:14<00:00, 66.91it/s]

Results


,Model,BIC,BIC 90% CI,BIC score diff,BIC score diff 90% CI,Xi²,% times preferred over simple,K-fold mean MSE,K-fold mean MSE 90% CI,Recent slope for closed models (Nx/year),Recent slope for closed models 90% CI,Recent slope for open models (Nx/year),Recent slope for open models 90% CI,Lead time (years),Lead time 90% CI
0,Simple,176.13,"[146.11, 193.56]",0.0,"[0.0, 0.0]",NaN,0%,0.22,"[0.17, 0.26]",4.99,"[4.23, 5.86]",3.51,"[3.08, 4.03]",1.43,"[1.12, 1.68]"


In [25]:
# Save results_df
regression_fname = f'compute_regression_analysis_{model_selection}_frontier={frontier_selection}_top{top_n}_cutoff={cutoff_date}.csv'
results_df.to_csv(os.path.join(results_dir, regression_fname), index=False)

# Save bootstrap_slopes as JSON
slopes_fname = f'bootstrap_slopes_{model_selection}_frontier={frontier_selection}_top{top_n}_cutoff={cutoff_date}.json'
with open(os.path.join(results_dir, slopes_fname), 'w') as f:
    json.dump(bootstrap_slopes, f, indent=4)

## Significant difference between regression slopes

### All data

In [26]:
df_filtered['date_float'] = datetime_to_float_year(df_filtered['date'])
open_df = df_filtered[df_filtered['category'] == 'Open']
closed_df = df_filtered[df_filtered['category'] == 'Closed']
regression_slope_t_test(open_df, closed_df, ['date_float'], 'log_flop', logy=False, adj_corr=True)

Slope 1: 0.55 (SE: 0.03)
Slope 2: 0.70 (SE: 0.04)
Correlation of residuals: 0.00
Test statistic: -3.08
p-value: 0.00


### Bootstrap distributions

In [27]:
open_slopes = bootstrap_slopes['Simple']['open']
closed_slopes = bootstrap_slopes['Simple']['closed']

In [28]:
# Plot a histogram of the slopes

# Create a DataFrame for the slopes
slopes_df = pd.DataFrame({
    'slope': np.log10(open_slopes + closed_slopes),
    'category': ['Open'] * len(open_slopes) + ['Closed'] * len(closed_slopes)
})

# Plot the histogram using plotly
fig = px.histogram(slopes_df, x='slope', color='category', barmode='overlay', 
                   title='Distribution of Bootstrap Slopes', 
                   labels={'Slope': 'Slope (OOMs/year)', 'count': 'Frequency'},
                   opacity=0.5, color_discrete_map={'Open': colors['open'], 'Closed': colors['closed']})

fig.update_layout(
    width=800,
    height=600,
)

fig.show()


In [29]:
# Shapiro-Wilk test for normality
_, p_value_open = stats.shapiro(np.log10(open_slopes))
print(f"Shapiro-Wilk test p-value for Open Models: {p_value_open}")

_, p_value_closed = stats.shapiro(np.log10(closed_slopes))
print(f"Shapiro-Wilk test p-value for Closed Models: {p_value_closed}")

# Anderson-Darling test for normality
result_open = stats.anderson(np.log10(open_slopes))
print(f"Anderson-Darling test statistic for Open Models: {result_open.statistic}")

result_closed = stats.anderson(np.log10(closed_slopes))
print(f"Anderson-Darling test statistic for Closed Models: {result_closed.statistic}")

Shapiro-Wilk test p-value for Open Models: 0.2713032758938014
Shapiro-Wilk test p-value for Closed Models: 0.33304016819936316
Anderson-Darling test statistic for Open Models: 0.5691435750875371
Anderson-Darling test statistic for Closed Models: 0.24622363716548534


In [30]:
# Use Mann-Whitney U test (if any test above rejects normality, p < 0.05)
statistic, p_value = stats.mannwhitneyu(np.log10(open_slopes), np.log10(closed_slopes))
print(f"Mann-Whitney U test: statistic={statistic}, p-value={p_value}")

# Use t-test otherwise
statistic, p_value = stats.ttest_ind(np.log10(open_slopes), np.log10(closed_slopes), equal_var=False)
print(f"t-test: statistic={statistic}, p-value={p_value}")

Mann-Whitney U test: statistic=3349.0, p-value=0.0
t-test: statistic=-84.58045503128825, p-value=0.0


In [31]:
np.percentile(np.log10(closed_slopes) - np.log10(open_slopes), [2.5, 97.5])

array([0.04476709, 0.26774494])

## Plot predictions

In [34]:
# Graph of the different model fits using plotly

model = 'simple'  # ['simple', 'kinked']
colors = {'open': '#1f77b4', 'closed': '#ff7f0e'}  # Using default plotly colors

# Parameters for the simple model
kink_count = 0
allow_discontinuities = False
same_intercepts = (False,)
same_slopes = (False,)

def plot_model(df, model_type, kink_count=1, allow_discontinuities=False):
    if model_type == 'simple':
        fit_result = fit_n_phase_exponential(df, 0, same_intercepts, same_slopes)
    else:
        fit_result = fit_n_phase_exponential(df, kink_count, allow_discontinuities, same_intercepts, same_slopes)

    df_open = df[df['category'] == 'Open']
    df_closed = df[df['category'] == 'Closed']

    fig = go.Figure()

    # Plot the original data points
    fig.add_trace(go.Scatter(
        x=df_open['date'], y=df_open['log_flop'],
        mode='markers', name='Open models',
        marker=dict(color=colors['open'], opacity=0.3)
    ))
    fig.add_trace(go.Scatter(
        x=df_closed['date'], y=df_closed['log_flop'],
        mode='markers', name='Closed models',
        marker=dict(color=colors['closed'], opacity=0.3)
    ))

    # Plot the fit lines
    date_grid = pd.date_range(start=df['date'].min(), end=df['date'].max(), freq='D')
    log_flop_open = fit_result.predict(pd.Series(date_grid), pd.Series(['Open'] * len(date_grid)))
    log_flop_closed = fit_result.predict(pd.Series(date_grid), pd.Series(['Closed'] * len(date_grid)))
    
    fig.add_trace(go.Scatter(
        x=date_grid, y=log_flop_open,
        mode='lines', name='Best Fit Line (Open)',
        line=dict(color=colors['open'])
    ))
    fig.add_trace(go.Scatter(
        x=date_grid, y=log_flop_closed,
        mode='lines', name='Best Fit Line (Closed)',
        line=dict(color=colors['closed'])
    ))

    # Add slope labels
    points = [df['date'].min()] + fit_result.break_points_dt + [df['date'].max()]
    for i in range(len(points) - 1):
        for category in ['open', 'closed']:
            mid = points[i] + (points[i+1] - points[i]) / 2
            y = fit_result.predict(pd.Series([mid]), pd.Series([category]))[0]
            fig.add_annotation(
                x=mid, y=y + 1.5 * (1 if category == 'closed' else -1),
                text=f'{10**fit_result.oom_year_slopes[category][i]:0.1f}x/year',
                showarrow=False,
                font=dict(size=12, color=colors[category])
            )

    # Plot horizontal line segment showing the lead time
    lead_time_years = calculate_lead_time(df, fit_result)
    end_date = df['date'].max()
    start_date = end_date - pd.DateOffset(days=int(lead_time_years * 365.25))
    y_value = fit_result.predict(pd.Series([end_date]), pd.Series(['Open']))[0]
    fig.add_shape(
        type="line",
        x0=start_date, y0=y_value, x1=end_date, y1=y_value,
        line=dict(color="black", width=1, dash="dash")
    )
    fig.add_annotation(
        x=(start_date + (end_date - start_date) * 0.5), y=y_value+0.25,
        text=f'Lead time: {lead_time_years*12:.0f} months',
        showarrow=False,
    )

    # Annotate some key models with text
    key_models = ['GPT-4', 'Llama 3.1-405B']
    for model_name in key_models:
        model_row = df_filtered[df_filtered['System'] == model_name]
        fig.add_annotation(
            x=model_row['date'].iloc[0], y=model_row['log_flop'].iloc[0],
            text=model_name,
            showarrow=False,
            font=dict(size=12, color='black'),
            xanchor='right', yanchor='bottom'
        )

    # Update layout
    fig.update_layout(
        template='plotly_white',
        width=800,
        height=400,
        title='Compute trends for open and closed models',
        xaxis_title='Model publication date',
        yaxis_title='Training compute (FLOP, log-scale)',
        legend_title='Model Category',
        legend=dict(
            x=0.7,
            y=0.1
        ),
        margin=dict(l=10, r=10, t=40, b=10),
        xaxis=dict(
            tickformat='%Y',
            dtick='M12',
        ),
        yaxis=dict(
            tickmode='array',
            tickvals=list(range(int(df['log_flop'].min()), int(df['log_flop'].max())+1)),
            ticktext=[f'10<sup>{i}</sup>' for i in range(int(df['log_flop'].min()), int(df['log_flop'].max())+1)]
        )
    )

    fname = f'compute_regression_{model_selection}_frontier={frontier_selection}_top{top_n}_cutoff={cutoff_date}_{model_type}_kinks={kink_count}'
    # save_plot(fig, results_dir, fname)

    fig.show()

plot_model(df_filtered, model, kink_count)

In [33]:
# Scatter plot of open and closed models using plotly
open_df = df_filtered[df_filtered['category'] == 'Open']
closed_df = df_filtered[df_filtered['category'] == 'Closed']

marker_to_org = {
    'bowtie': 'Meta',
    'cross': 'Google',
    'hexagon-open': 'OpenAI',
    'star': 'Anthropic',
    'square': 'Microsoft',
    'circle': 'Other',
}
closed_added_to_legend = defaultdict(bool)
open_added_to_legend = defaultdict(bool)

fig = go.Figure()
for org in df_filtered['Organization'].unique():
    open_df_org = open_df[open_df['Organization'] == org]
    closed_df_org = closed_df[closed_df['Organization'] == org]
    if any([kw in org.lower() for kw in ['meta', 'facebook']]):
        marker = 'bowtie'
    elif any([kw in org.lower() for kw in ['google', 'deepmind']]):
        marker = 'cross'
    elif any([kw in org.lower() for kw in ['openai']]):
        marker = 'hexagon-open'
    elif any([kw in org.lower() for kw in ['anthropic']]):
        marker = 'star'
    elif any([kw in org.lower() for kw in ['microsoft']]):
        marker = 'square'
    else:
        marker = 'circle'
    fig.add_trace(go.Scatter(
        x=open_df_org['date'],
        y=open_df_org['log_flop'],
        text=open_df_org['System'],
        mode='markers',
        name=marker_to_org[marker] + ', open',
        showlegend=not open_added_to_legend[marker],
        marker=dict(
            color=colors['open'],
            opacity=0.5,
            symbol=marker
        )
    ))
    fig.add_trace(go.Scatter(
        x=closed_df_org['date'],
        y=closed_df_org['log_flop'],
        text=closed_df_org['System'],
        mode='markers',
        name=marker_to_org[marker] + ', closed',
        showlegend=not closed_added_to_legend[marker],
        marker=dict(
            color=colors['closed'],
            opacity=0.5,
            symbol=marker
        )
    ))
    if len(closed_df_org) > 0:
        closed_added_to_legend[marker] = True
    if len(open_df_org) > 0:
        open_added_to_legend[marker] = True

# Axis titles
fig.update_layout(xaxis_title='Model publication date')
fig.update_layout(yaxis_title='Training compute (FLOP, log-scale)')

# Format the y-axis labels as 10^N
yvals = list(range(20, 27))
fig.update_yaxes(
    tickmode = 'array',
    tickvals = yvals,
    ticktext = [f'10<sup>{x}</sup>' for x in yvals],
    # ticks="",
    # tickfont=dict(size=20)
)

# Legend title
fig.update_layout(legend_title='Organization, access')

# Margins
fig.update_layout(margin=dict(l=10, r=10, t=40, b=10))

# plotly-white
fig.update_layout(template='plotly_white')

# Sizing
fig.update_layout(
    width=600,
    height=400,
    title='Open and closed models by organization'
)

# Save
save_plot(fig, results_dir, f'compute_open_closed_by_org_{model_selection}_frontier={frontier_selection}_top{top_n}_cutoff={cutoff_date}')

fig.show()